# Project: N-Gram Sentence Generator

# Krishna Menon I045
# Aditya Rana I059
# Yash Kothari I037

## Problem Statement

**Objective:** To implement an n-gram-based language model (LM) capable of generating coherent sentences. This project involves preprocessing a text corpus, building bigram (n=2), trigram (n=3), and 4-gram (n=4) models with Laplace smoothing, and generating new sentences from a two-word prompt.

**Evaluation:** The models will be evaluated quantitatively using **perplexity** on a held-out test set and qualitatively by analyzing the **fluency** and **coherence** of the generated sentences.

In [1]:
import nltk
import numpy as np
import pandas as pd
from collections import defaultdict
import random


nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('punkt_tab')

print("Libraries imported and NLTK data downloaded.")

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Libraries imported and NLTK data downloaded.


In [2]:
from nltk.corpus import gutenberg
raw_sentences = list(gutenberg.sents('austen-emma.txt'))

print(f"Total sentences loaded: {len(raw_sentences)}")
print("Example sentence:", raw_sentences[50])

Total sentences loaded: 7752
Example sentence: ['But', 'James', 'will', 'not', 'like', 'to', 'put', 'the', 'horses', 'to', 'for', 'such', 'a', 'little', 'way', ';--', 'and', 'where', 'are', 'the', 'poor', 'horses', 'to', 'be', 'while', 'we', 'are', 'paying', 'our', 'visit', '?"']


In [3]:
split_index = int(0.8 * len(raw_sentences))
train_raw_sents = raw_sentences[:split_index]
test_raw_sents = raw_sentences[split_index:]

print(f"Training sentences: {len(train_raw_sents)}")
print(f"Testing sentences: {len(test_raw_sents)}")

Training sentences: 6201
Testing sentences: 1551


In [4]:
def build_vocab(sentences):
    vocab = set()
    for sent in sentences:
        for word in sent:
            vocab.add(word.lower())
    vocab.add('<UNK>')
    vocab.add('<s>')
    vocab.add('</s>')
    return vocab

vocabulary = build_vocab(train_raw_sents)
V = len(vocabulary)
print(f"Vocabulary size (V): {V}")

Vocabulary size (V): 6595


In [5]:
def preprocess(sentences, vocab_set):
    processed_sentences = []
    n_pad = 3 #

    for sent in sentences:
        tokens = []
        for word in sent:
            word_low = word.lower()
            if word_low in vocab_set:
                tokens.append(word_low)
            else:
                tokens.append('<UNK>')
        padding = ["<s>"] * n_pad
        processed_sentences.append(padding + tokens + ["</s>"])

    return processed_sentences

train_data = preprocess(train_raw_sents, vocabulary)
test_data = preprocess(test_raw_sents, vocabulary)

print("Example preprocessed sentence (train_data[50]):")
print(train_data[50])

Example preprocessed sentence (train_data[50]):
['<s>', '<s>', '<s>', 'but', 'james', 'will', 'not', 'like', 'to', 'put', 'the', 'horses', 'to', 'for', 'such', 'a', 'little', 'way', ';--', 'and', 'where', 'are', 'the', 'poor', 'horses', 'to', 'be', 'while', 'we', 'are', 'paying', 'our', 'visit', '?"', '</s>']


In [7]:
class NGramModel:
    def __init__(self, n, k, vocab):
        self.n = n
        self.k = k
        self.vocab = vocab
        self.V = len(vocab)
        self.n_gram_counts = defaultdict(int)
        self.prefix_counts = defaultdict(int)

    def fit(self, sentences):
        print(f"Training {self.n}-gram model...")
        start_idx = 3 - (self.n - 1)
        for sentence in sentences:
            tokens = sentence[start_idx:]
            for i in range(len(tokens) - self.n + 1):
                n_gram = tuple(tokens[i : i + self.n])
                prefix = tuple(tokens[i : i + self.n - 1])
                self.n_gram_counts[n_gram] += 1
                self.prefix_counts[prefix] += 1
        print("Training complete.")

    def get_prob(self, n_gram):
        """Calculates the smoothed probability of a single n-gram."""
        prefix = n_gram[:-1]
        numerator = self.n_gram_counts[n_gram] + self.k
        denominator = self.prefix_counts[prefix] + (self.k * self.V)
        if denominator == 0:
            return 1 / self.V

        return numerator / denominator

    def get_next_word_probs(self, prefix_words):
        """Returns a probability distribution for all words in vocab given a prefix."""
        prefix = tuple(prefix_words[-(self.n-1):])
        probs = {}
        denominator = self.prefix_counts[prefix] + (self.k * self.V)
        for word in self.vocab:
            n_gram = prefix + (word,)
            numerator = self.n_gram_counts[n_gram] + self.k
            probs[word] = numerator / denominator

        return probs

In [8]:
k_val = 1

model_2 = NGramModel(n=2, k=k_val, vocab=vocabulary)
model_2.fit(train_data)

model_3 = NGramModel(n=3, k=k_val, vocab=vocabulary)
model_3.fit(train_data)

model_4 = NGramModel(n=4, k=k_val, vocab=vocabulary)
model_4.fit(train_data)

Training 2-gram model...
Training complete.
Training 3-gram model...
Training complete.
Training 4-gram model...
Training complete.


In [9]:
def generate_sentence(model, start_words, max_len=12):
    n = model.n
    sentence = [w.lower() for w in start_words]
    context = (["<s>"] * 3) + [w.lower() for w in start_words]
    for _ in range(max_len - len(start_words)):
        prefix = tuple(context[-(n-1):])
        word_probs = model.get_next_word_probs(prefix)
        words = list(word_probs.keys())
        probs = list(word_probs.values())
        probs_sum = sum(probs)
        normalized_probs = [p / probs_sum for p in probs]
        next_word = np.random.choice(words, p=normalized_probs)
        if next_word == "</s>":
            break
        sentence.append(next_word)
        context.append(next_word)

    return " ".join(sentence) + "."

In [10]:
start_prompt = ["Once", "upon"]
num_sentences = 5

print(" Bigram Model (n=2) ")
for i in range(num_sentences):
    print(f"{i+1}: {generate_sentence(model_2, start_prompt)}")

print("\n Trigram Model (n=3)  ")
for i in range(num_sentences):
    print(f"{i+1}: {generate_sentence(model_3, start_prompt)}")

print("\n 4-gram Model (n=4)  ")
for i in range(num_sentences):
    print(f"{i+1}: {generate_sentence(model_4, start_prompt)}")

 Bigram Model (n=2) 
1: once upon lame wherever walk caprice bounds cellery apprehensive throat bad whenever.
2: once upon fellow miss regulate unexceptionably grown pitcher regards pursued bangs doubled.
3: once upon stilton trimming _times_ respective chief accompanying comprehend undistinguishing peculiar inconsiderable.
4: once upon haunting maintained wakefield replies wholly next reproof recommendations uninterruptedly horror.
5: once upon mount effectually cramp occurred spreading knife nearer check answering ?).

 Trigram Model (n=3)  
1: once upon _way_ spurn entrapped temper owner right degradation opportunity various propensity.
2: once upon bella mystery averted errand supposed unmanageable tear marked regular imagines.
3: once upon boy cox disturb appears undertaking staid awes forestalling cheerfuller apiece.
4: once upon disturbance envy passionately ardent throws ?-- hurt best little chained.
5: once upon proverb state washed partridge work seven painful formal pales ac

In [11]:
def calculate_perplexity(model, test_sentences):
    print(f"Calculating perplexity for n={model.n}...")
    n = model.n
    total_log_prob = 0
    total_words = 0
    start_idx = 3 - (n - 1)
    for sentence in test_sentences:
        tokens = sentence[start_idx:]
        total_words += len(tokens) - (n - 1)
        for i in range(n - 1, len(tokens)):
            n_gram = tuple(tokens[i - n + 1 : i + 1])
            prob = model.get_prob(n_gram)
            total_log_prob += np.log2(prob)


    avg_log_likelihood = total_log_prob / total_words
    perplexity = np.power(2, -avg_log_likelihood)
    return perplexity

In [14]:
ppl_2 = calculate_perplexity(model_2, test_data)
ppl_3 = calculate_perplexity(model_3, test_data)
ppl_4 = calculate_perplexity(model_4, test_data)

results = {
    "Model": ["Bigram (n=2)", "Trigram (n=3)", "4-gram (n=4)"],
    "Perplexity (on Test Set)": [ppl_2, ppl_3, ppl_4]
}

results_df = pd.DataFrame(results)
results_df.set_index("Model", inplace=True)
print(results_df.to_markdown(floatfmt=".2f"))

Calculating perplexity for n=2...
Calculating perplexity for n=3...
Calculating perplexity for n=4...
| Model         |   Perplexity (on Test Set) |
|:--------------|---------------------------:|
| Bigram (n=2)  |                     738.85 |
| Trigram (n=3) |                    3156.12 |
| 4-gram (n=4)  |                    4707.38 |


## **Quantitative (Perplexity):**
 We expect perplexity to decrease as n increases. The 4-gram model should have the lowest perplexity, followed by the trigram, and then the bigram. This is because models with larger n can use more context to predict the next word, making them more accurate. The 4-gram model's predictions P(w | w-3, w-2, w-1) are much more informed than the bigram's  P(w | w-1).


## **Qualitative (Fluency):**
In our assignment, we learned that sentences generated by our model corresponded with the perplexity scores that we were observing. The result of the bigram model was gibberish. The trigram form was comprised of a skeleton of good grammar, and yet of no actual connection.


## **Limitations of N-Gram Models:**
 When we pushed up to 4-gram, the phrases became the most fluent and locally coherent which really was the best of them being able to use a larger context window. The possible n-grams increase exponentially (Vn). The majority of 4 grams such as once upon a time never appear in our training corpus even when it is massive. That means the count is zero. A non-zero probability will be obtained with Laplace smoothing, but it is more of a conjecture. The reason is that, with large n (larger than 5 or 6) you will find perplexity again rising, because at this point the n-gram model will no longer be able to remember the long-term context, which is the entire conversation. A 4 gram cannot grasp what was being spoken 10 words ago. That is why it is unable to produce really coherent paragraph. It does not pick semantics and dependencies. To give a few examples, “The man who lives down the street... is... may drop the subject man and suppose that it is speaking about the street.


## **Conclusion:**
 This project has managed to demonstrate the construction of n-gram language models successfully. We had a definite trade-off: increasing n to 4 resulted in better quality (less perplexity, higher fluency), because more context was captured, but deteriorated sparsity and computational cost. The experiment introduces the major drawbacks of n-gram models and is a step towards more sophisticated methods, such as neural language models (RNNs, LSTMs, Transformers), which are capable of representing long-range interactions, as well as acquiring distributed representations of words.